In [3]:
import pandas as pd
import numpy as np
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

In [5]:
queries = {'con1':[200,500], 'con2':[400,300], 'con3':[700,0]}

for query in queries:
       # Analyze partial opa blast results

       # Load in blast results for partial opas
       blast = pd.read_csv("../../results/blast_partial_opa/opa_" + query + "_blast.tab", sep = '\t', skiprows = 5, skipfooter = 1, header = None, names = ['query', 'subject', 'percent_identity', 'alignment_length', 'mismatches', 'gap_opens', 'query_start', 'query_end', 'subject_start', 'subject_end', 'evalue', 'bit_score'])

       # Load in opa metadata
       metadata = pd.read_csv("../../results/opa_metadata_locus.csv", index_col = 0)

       blast.sort_values('subject_start', inplace = True, ignore_index = True)
       metadata.sort_values('start', inplace = True, ignore_index = True)
       metadata = metadata[['strand', 'start_cr', 'stop_cr', 'start_term',
              'stop_term', 'start', 'stop', 'n_terminus', 'id', 'in_frame', 'strain', 'locus']]
       metadata['start'] = metadata['start'].astype('int')

       # Merge blast results with full length opa
       merged = pd.merge_asof(blast, metadata, left_on = 'subject_start', right_on = 'start', left_by = 'subject', right_by = 'strain', direction = 'nearest', tolerance = 1200)
       blast_only = merged[merged['start'].isna()].sort_values(['subject', 'subject_start'])

       # Get strand info
       blast_only.loc[blast_only['subject_end']>blast_only['subject_start'],'strand']=1
       blast_only.loc[blast_only['subject_end']<blast_only['subject_start'],'strand']=-1
       blast_only['strand']=blast_only['strand'].astype('int')

       # Get approx start/stop info of gene
       upstream = queries[query][0]
       downstream = queries[query][1]
       blast_only.loc[blast_only['strand']==1, 'approx_start'] = blast_only[blast_only['strand']==1]['subject_start']-upstream
       blast_only.loc[blast_only['strand']==1, 'approx_stop'] = blast_only[blast_only['strand']==1]['subject_end']+downstream

       blast_only.loc[blast_only['strand']==-1, 'approx_start'] = blast_only[blast_only['strand']==-1]['subject_start']+upstream
       blast_only.loc[blast_only['strand']==-1, 'approx_stop'] = blast_only[blast_only['strand']==-1]['subject_end']-downstream

       # Get the sequences for the blast opas that were not found in the metadata
       path = '../../results/assemblies_shifted/'
       blast_only_records = []
       i = 0
       for i, row in blast_only.iterrows():
              strain = row['subject']
              start = int(row['approx_start'])
              stop = int(row['approx_stop'])
              strand = row['strand']
              
              record = list(SeqIO.parse(path + strain + '.fa', "fasta"))[0]
              if strand == 1:
                     blast_only_records.append(SeqRecord(record.seq[start-100:stop+100], id = strain + '_' + str(start) + '_' + str(stop)))
              elif strand == -1:
                     blast_only_records.append(SeqRecord(record.seq[stop-100:start+100].reverse_complement(), id = strain + '_' + str(start) + '_' + str(stop)))

       # Add FA1090 sequences, with extra sequence before and after to test for deletions
       for i, row in metadata[metadata['strain']=='FA1090'].iterrows():
              strain = row['strain']
              start = int(row['start'])
              stop = int(row['stop'])
              strand = row['strand']
              
              record = list(SeqIO.parse(path + 'FA1090.fa', "fasta"))[0]
              if strand == 1: 
                     blast_only_records.append(SeqRecord(record.seq[start:stop], id = row['id']))
              elif strand == -1:
                     blast_only_records.append(SeqRecord(record.seq[start:stop].reverse_complement(), id = row['id']))
       
       SeqIO.write(blast_only_records, '../../results/blast_partial_opa/opa_' + query + '_blast_only_with_FA1090.fa', 'fasta')

/tmp/ipykernel_236738/1725220249.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  blast = pd.read_csv("../../results/blast_partial_opa/opa_" + query + "_blast.tab", sep = '\t', skiprows = 5, skipfooter = 1, header = None, names = ['query', 'subject', 'percent_identity', 'alignment_length', 'mismatches', 'gap_opens', 'query_start', 'query_end', 'subject_start', 'subject_end', 'evalue', 'bit_score'])
/tmp/ipykernel_236738/1725220249.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  blast = pd.read_csv("../../results/blast_partial_opa/opa_" + query + "_blast.tab", sep = '\t', skiprows = 5, skipfooter = 1, header = None, names = ['query', 'subject', 'percent_identity', 'alignment_length', 'mismatches', 'gap_opens', 'query_start', 'query_end', 'subject_sta

In [34]:
# Find genes that our method found that BLAST did not find

our_method_only_opas = {}
for query in queries:
        # Load in blast results for partial opas
        blast = pd.read_csv("../../results/blast_partial_opa/opa_"+query+"_blast.tab", sep = '\t', skiprows = 5, skipfooter = 1, header = None, names = ['query', 'subject', 'percent_identity', 'alignment_length', 'mismatches', 'gap_opens', 'query_start', 'query_end', 'subject_start', 'subject_end', 'evalue', 'bit_score'])

        # Load in opa metadata
        metadata = pd.read_csv("../../results/opa_metadata_locus.csv", index_col = 0)

        blast.sort_values('subject_start', inplace = True, ignore_index = True)
        metadata.sort_values('start', inplace = True, ignore_index = True)
        metadata = metadata[['strand', 'start_cr', 'stop_cr', 'start_term',
                'stop_term', 'start', 'stop', 'n_terminus', 'id', 'in_frame', 'strain', 'locus']]
        metadata['start'] = metadata['start'].astype('int')

        # Merge blast results with full length opa
        merged = pd.merge_asof(metadata, blast, right_on = 'subject_start', left_on = 'start', right_by = 'subject', left_by = 'strain', direction = 'nearest', tolerance = 1200)
        our_method_only = merged[merged['subject_start'].isna()].sort_values(['subject', 'subject_start'])

        our_method_only_opas[query]=our_method_only['id'].values

# Get the opas that were not found by any of the 3 blast searches
our_method_only_opas_overall = set(our_method_only_opas['con2']).intersection(set(our_method_only_opas['con1'])).intersection(set(our_method_only_opas['con3']))
print('Num opa not found by blast search: ', len(our_method_only_opas_overall))
print(our_method_only_opas_overall)

Num opa not found by blast search:  8
{'GCGS0864_opa_2', 'RIVM0610_opa_2', 'WHO_Y_2024_opa_2', 'WHO_H_2024_opa_2', '1130991_opa_2', 'RIVM0640_opa_2', 'FQ48_opa_2', '1081168_opa_2'}


/tmp/ipykernel_236738/1849525997.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  blast = pd.read_csv("../../results/blast_partial_opa/opa_"+query+"_blast.tab", sep = '\t', skiprows = 5, skipfooter = 1, header = None, names = ['query', 'subject', 'percent_identity', 'alignment_length', 'mismatches', 'gap_opens', 'query_start', 'query_end', 'subject_start', 'subject_end', 'evalue', 'bit_score'])
/tmp/ipykernel_236738/1849525997.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  blast = pd.read_csv("../../results/blast_partial_opa/opa_"+query+"_blast.tab", sep = '\t', skiprows = 5, skipfooter = 1, header = None, names = ['query', 'subject', 'percent_identity', 'alignment_length', 'mismatches', 'gap_opens', 'query_start', 'query_end', 'subject_start', 'su